In [1]:
import json, requests, csv
from contextlib import closing
from datetime import timedelta, date, datetime
import dateparser #helpful for ambigious dates

In [2]:
Batch_ID= 2995585
API_token =  {'Captricity-API-Token': '435c308483e040fe9928418703216584'}
BASE_URL = 'shreddr.captricity.com' 

In [3]:
def get_batch_info(batch_id, headers):
    url = 'https://{}/api/v1/batch/{}'.format(BASE_URL, batch_id)
    batch = requests.get(url, headers=headers)
    return json.loads(batch.text)

 - > get list of child Batches from parent Batch
 

In [4]:
Parent_batch = get_batch_info(Batch_ID,API_token)

In [5]:
# Parent_batch

   - > for each child Batch 
        - > get the child Batch info
        - > if it is not the reject Batch
        

In [6]:
def get_children_job_info(parent_batch):
    """
    Returns list of job ids from valid children batches
    """
    job_list = []
    for ids in Parent_batch['children_ids']:
        child_branch = get_batch_info(ids, API_token)
        if not child_branch['reject_reasons']: #verify if it's a reject batch pass if so, else return  jobs id
            job_list.extend(child_branch['job_ids'])
        else:
            pass
            
    return job_list

In [7]:
job_list = get_children_job_info(Parent_batch)
# job_list[0]

        - > pull the associated Job's output CSV
                - > Question- are we always assuming one jobs ID? or should I account for multiple?

In [8]:
def get_job_info(job_id, headers):
    url = 'https://{}/api/v1/job/{}/csv/'.format(BASE_URL, job_id)
    job = requests.get(url, headers=headers)
    job_content = job.content.decode('utf-8')
    cr = csv.reader(job_content.splitlines(), delimiter=',')
    my_list = list(cr)
    return my_list
# job.text

In [9]:
job_text = get_job_info(job_list[0], API_token)
job_text[0][4]
# job_text[0][7]
# job_text[0][8]

'patient_dob'

In [10]:
print('patient_dob:')
for row in job_text[1:]:
    print(row[4])
print('dos_from:')
for row in job_text[1:]:
    print(row[7])
print('dos_to:')
for row in job_text[1:]:
    print(row[8])

patient_dob:
06/20/1979
072859
--blank--
--blank--
12221978
dos_from:
05/30/2000
04/23/2018
0902055
08 19 80
02/12/2018
dos_to:
05 30 2001
042418
--blank--
09/20/1999
04/12/2018


In [11]:
for row in job_text[1:]:
    row[4]=convert_date(row[4])
    row[7]=convert_date(row[7])
    row[8]=convert_date(row[8])    

NameError: name 'convert_date' is not defined

In [ ]:
print('patient_dob:')
for row in job_text[1:]:
    print(row[4])
print('dos_from:')
for row in job_text[1:]:
    print(row[7])
print('dos_to:')
for row in job_text[1:]:
    print(row[8])

In [ ]:
def format_date(old_date):
    if old_date > old_date.today():
        old_date -= timedelta(days=365.25*100)
    else:
        pass
    return '{}-{:02d}-{:02d}'.format(old_date.year, old_date.month, old_date.day)

In [33]:
#Simple checker
def convert_date(try_date):
    #Can always add in more potential date patterns
    date_patterns = ["%m/%d/%Y", "%m%d%y", '%m%d%Y', '%m%d%y' '%m %d %Y','%Y-%m-%d', '%m %d %y', '%m %d %Y']
    if try_date == '--blank--':
        return '--blank--'
    else:   
        for pattern in date_patterns:
            try:
                datetime.strptime(try_date, pattern).date()  
                return format_date(datetime.strptime(try_date, pattern).date())
            except Exception as e:
                print(e)
                pass
    return '--impossible--'

In [34]:
convert_date('Nick')

time data 'Nick' does not match format '%m/%d/%Y'
time data 'Nick' does not match format '%m%d%y'
time data 'Nick' does not match format '%m%d%Y'
redefinition of group name 'm' as group 4; was group 1 at position 82
time data 'Nick' does not match format '%Y-%m-%d'
time data 'Nick' does not match format '%m %d %y'
time data 'Nick' does not match format '%m %d %Y'


'--impossible--'

In [ ]:
job_text

In [ ]:
def write_to_csv(csv_rows):
    writer = csv.writer(open("captricity2_csv.csv", 'w'))
    for row in csv_rows:
        writer.writerow(row)

In [ ]:
write_to_csv(job_text)